In [1]:
import lxml
import newspaper
import mitie
import pandas as pd

In [ ]:
usatoday = newspaper.build("https://www.usatoday.com/")
washpost = newspaper.build("https://www.washingtonpost.com/")
#atlantic = newspaper.build("https://www.theatlantic.com/")
newyorker = newspaper.build("https://www.newyorker.com/")

In [ ]:
articles = []
for i in range(10):
    articles.append(usatoday.articles[i])
    articles.append(washpost.articles[i])
    #articles.append(atlantic.articles[i])
    articles.append(newyorker.articles[i])

In [ ]:
texts = []
for article in articles:
    article.download()
    article.parse()
    texts.append((article.title, article.text))

In [ ]:
import sys, os

In [ ]:
sys.path.append(os.getenv('HOME') + '/Packages/MITIE/mitielib')

In [ ]:
#words = article.text.split(' ')

In [ ]:
tokens = [mitie.tokenize(text[1]) for text in texts]


In [ ]:
ner = mitie.named_entity_extractor(os.getenv('HOME') + '/Packages/MITIE/MITIE-models/english/ner_model.dat')

In [ ]:
entities_list = [ner.extract_entities(token) for token in tokens]

In [ ]:
print(len(entities_list[0]))
print(entities_list[0])

In [ ]:
entity_texts = []
for index, entities in enumerate(entities_list):
    token = tokens[index]
    temp = []
    for e in entities:
        width = e[0]
        tag = e[1]
        score = e[2]
        score_text = "{:0.3f}".format(score)
        entity_text = " ".join(token[i].decode() for i in width)
        temp.append(entity_text)
        #print("   Score: " + score_text + ": " + tag + " " + entity_text)
        
    entity_texts.append(temp)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tf = [" ".join(t) for t in entity_texts]

In [ ]:
vect = TfidfVectorizer()

In [ ]:
tfidf = vect.fit_transform(tf)

In [ ]:
terms = vect.get_feature_names()

In [ ]:
pairwise_sim = (tfidf * tfidf.T).A

In [ ]:
from sklearn.cluster import SpectralClustering

In [ ]:
spectral = SpectralClustering(eigen_solver='lobpcg', affinity='precomputed')

In [ ]:
spectral.fit(pairwise_sim)

In [ ]:
# print("Terms per cluster:")
# for i in __builtins__.range(8):
#     print("Cluster %d:" % i),
#     T=t[spectral.labels_==i].indices
#     for ind in T:
#         print(terms[ind])
#     print("")

In [ ]:
spectral.labels_

In [ ]:
mydict = {'title': [text[0] for text in texts], 'label':spectral.labels_}
df = pd.DataFrame(mydict)

In [ ]:
df[df.label == 3].title.tolist()